# Mask R-CNN - Train modified model on old shapes dataset

### the modified model does not include any mask related heads or losses 



In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import tensorflow as tf
import keras.backend as KB
import numpy as np


##-------------------------------------------------------------------------------------------
##
## Alternate training of FCN
##  - start with trained MRCNN model, pass predcitions over to FCN
##
##-------------------------------------------------------------------------------------------

import os
import sys
import math
import gc
import time
import numpy as np
import argparse
import platform
import tensorflow as tf
import keras
import keras.backend as KB
import platform

sys.path.append('../')

import mrcnn.model_mod2  as mrcnn_modellib
import mrcnn.model_fcn  as fcn_modellib

import mrcnn.visualize  as visualize
import mrcnn.new_shapes     as shapes
from mrcnn.config       import Config
from mrcnn.dataset      import Dataset 
from mrcnn.utils        import log, stack_tensors, stack_tensors_3d
from mrcnn.datagen      import data_generator, load_image_gt
from mrcnn.callbacks    import get_layer_output_1,get_layer_output_2
# from mrcnn.visualize    import plot_gaussian
# from mrcnn.prep_notebook import prep_oldshapes_train, load_model

import pprint
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4,threshold=1000, suppress = True)
syst = platform.system()
 


##------------------------------------------------------------------------------------
## process input arguments
##  example:
##           train-shapes_gpu --epochs 12 --steps-in-epoch 7 --last_epoch 1234 --logs_dir mrcnn_logs
##------------------------------------------------------------------------------------
# Parse command line arguments
parser = argparse.ArgumentParser(description='Train Mask R-CNN on MS COCO.')
# parser.add_argument("command",
                    # metavar="<command>",
                    # help="'train' or 'evaluate' on MS COCO")
# parser.add_argument('--dataset', required=True,
                    # metavar="/path/to/coco/",
                    # help='Directory of the MS-COCO dataset')
# parser.add_argument('--limit', required=False,
                    # default=500,
                    # metavar="<image count>",
                    # help='Images to use for evaluation (defaults=500)')
                    
parser.add_argument('--model', required=False,
                    default='last',
                    metavar="/path/to/weights.h5",
                    help="MRCNN model weights file: 'coco' , 'init' , or Path to weights .h5 file ")

parser.add_argument('--fcn_model', required=False,
                    default='last',
                    metavar="/path/to/weights.h5",
                    help="FCN model weights file: 'init' , or Path to weights .h5 file ")

parser.add_argument('--logs_dir', required=True,
                    default='mrcnn_logs',
                    metavar="/path/to/logs/",
                    help='Logs and checkpoints directory (default=logs/)')
                    
parser.add_argument('--last_epoch', required=False,
                    default=0,
                    metavar="<last epoch ran>",
                    help='Identify last completed epcoh for tensorboard continuation')

parser.add_argument('--lr', required=False,
                    default=0.001,
                    metavar="<learning rate>",
                    help='Learning Rate (default=0.001)')

parser.add_argument('--epochs', required=False,
                    default=3,
                    metavar="<epochs to run>",
                    help='Number of epochs to run (default=3)')
                    
parser.add_argument('--steps_in_epoch', required=False,
                    default=1,
                    metavar="<steps in each epoch>",
                    help='Number of batches to run in each epochs (default=5)')
                    
parser.add_argument('--batch_size', required=False,
                    default=5,
                    metavar="<batch size>",
                    help='Number of data samples in each batch (default=5)')                    
                    
# args = parser.parse_args()
args = parser.parse_args("--epochs 100 --steps_in_epoch 128  --last_epoch 0  --batch_size 8 --lr 0.01 --logs_dir train_fcn_alt --model /home/kbardool/models/newshape_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 --fcn_model init".split())

Using TensorFlow backend.


In [2]:
pp.pprint(args)
print()
print("Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))
print("Model              :   ", args.model)
# print("Dataset: ", args.dataset)
# print("Logs:    ", args.logs)
# print("Limit:   ", args.limit)
print("Epochs to run      :   ", args.epochs)
print("Steps in each epoch:   ", args.steps_in_epoch)
print('OS Platform        :   ', syst)

##------------------------------------------------------------------------------------
## setup project directories
#---------------------------------------------------------------------------------
# # Root directory of the project 
# MODEL_DIR    :    Directory to save logs and trained model
# COCO_MODEL_PATH  : Path to COCO trained weights
#---------------------------------------------------------------------------------
if syst == 'Windows':
    # WINDOWS MACHINE ------------------------------------------------------------------
    ROOT_DIR          = "E:\\"
    MODEL_PATH        = os.path.join(ROOT_DIR    , "models")
    DATASET_PATH      = os.path.join(ROOT_DIR    , 'MLDatasets')
    MODEL_DIR         = os.path.join(MODEL_PATH  , args.logs_dir)
    COCO_MODEL_PATH   = os.path.join(MODEL_PATH  , "mask_rcnn_coco.h5")
    DEFAULT_LOGS_DIR  = os.path.join(MODEL_PATH  , args.logs_dir) 
    COCO_DATASET_PATH = os.path.join(DATASET_PATH, "coco2014")
    RESNET_MODEL_PATH = os.path.join(MODEL_PATH  , "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")
    VGG16_MODEL_PATH  = os.path.join(MODEL_PATH  , "fcn_vgg16_weights_tf_dim_ordering_tf_kernels.h5")
    
elif syst == 'Linux':
    # LINUX MACHINE ------------------------------------------------------------------
    ROOT_DIR          = os.getcwd()
    MODEL_PATH        = os.path.expanduser('~/models')
    DATASET_PATH      = os.path.expanduser('~/MLDatasets')
    MODEL_DIR         = os.path.join(MODEL_PATH  , args.logs_dir)
    COCO_MODEL_PATH   = os.path.join(MODEL_PATH  , "mask_rcnn_coco.h5")
    COCO_DATASET_PATH = os.path.join(DATASET_PATH, "coco2014")
    DEFAULT_LOGS_DIR  = os.path.join(MODEL_PATH  , args.logs_dir)
    RESNET_MODEL_PATH = os.path.join(MODEL_PATH  , "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")
    VGG16_MODEL_PATH  = os.path.join(MODEL_PATH  , "fcn_vgg16_weights_tf_dim_ordering_tf_kernels.h5")
else :
    raise Error('unreconized system  '      )


    
# class InferenceConfig(CocoConfig):
    # # Set batch size to 1 since we'll be running inference on
    # # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    # GPU_COUNT = 1
    # IMAGES_PER_GPU = 1
    # DETECTION_MIN_CONFIDENCE = 0
# config = InferenceConfig()
##------------------------------------------------------------------------------------
## Build configuration object 
##------------------------------------------------------------------------------------
mrcnn_config                    = shapes.NewShapesConfig()
mrcnn_config.BATCH_SIZE         = int(args.batch_size)                  # Batch size is 2 (# GPUs * images/GPU).
mrcnn_config.IMAGES_PER_GPU     = int(args.batch_size)                  # Must match BATCH_SIZE
mrcnn_config.STEPS_PER_EPOCH    = int(args.steps_in_epoch)
mrcnn_config.LEARNING_RATE      = float(args.lr)
                          
mrcnn_config.EPOCHS_TO_RUN      = int(args.epochs)
mrcnn_config.FCN_INPUT_SHAPE    = mrcnn_config.IMAGE_SHAPE[0:2]
mrcnn_config.LAST_EPOCH_RAN     = int(args.last_epoch)
mrcnn_config.WEIGHT_DECAY       = 2.0e-4
mrcnn_config.VALIDATION_STEPS   = 100
mrcnn_config.REDUCE_LR_FACTOR   = 0.5
mrcnn_config.REDUCE_LR_COOLDOWN = 30
mrcnn_config.REDUCE_LR_PATIENCE = 40
mrcnn_config.EARLY_STOP_PATIENCE= 80
mrcnn_config.MIN_LR             = 1.0e-10
# mrcnn_config.display() 

##------------------------------------------------------------------------------------
## Build shape dataset        
##------------------------------------------------------------------------------------
# Training dataset
# generate 500 shapes 
dataset_train = shapes.NewShapesDataset(mrcnn_config)
dataset_train.load_shapes(10000)
dataset_train.prepare()

# Validation dataset
dataset_val = shapes.NewShapesDataset(mrcnn_config)
dataset_val.load_shapes(2500)
dataset_val.prepare()

print('dataset gen complete')

Namespace(batch_size='8', epochs='100', fcn_model='init', last_epoch='0', logs_dir='train_fcn_alt', lr='0.01', model='/home/kbardool/models/newshape_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5', steps_in_epoch='128')

Tensorflow Version: 1.6.0   Keras Version : 2.1.4 
Model              :    /home/kbardool/models/newshape_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5
Epochs to run      :    100
Steps in each epoch:    128
OS Platform        :    Linux
dataset gen complete


In [3]:
##------------------------------------------------------------------------------------
## Build Mask RCNN Model in TRAINFCN mode
##------------------------------------------------------------------------------------

try :
    del mrcnn_model
    gc.collect()
except: 
    pass
KB.clear_session()
mrcnn_model = mrcnn_modellib.MaskRCNN(mode="trainfcn", config=mrcnn_config, model_dir=MODEL_DIR)

print(' COCO Model Path       : ', COCO_MODEL_PATH)
print(' Checkpoint folder Path: ', MODEL_DIR)
print(' Model Parent Path     : ', MODEL_PATH)
# print(model.find_last())


>>> Initialize MRCNN model, mode:  trainfcn
    set_log_dir(): Checkpoint path/filename set to          : /home/kbardool/models/train_fcn_alt/shapes20180709T1457/mask_rcnn_shapes_{epoch:04d}.h5 
    set_log_dir(): Last completed epoch (self.epoch) set to : 0 

>>> Resnet Graph 
     Input_image shape : (?, 128, 128, 3)
     After ZeroPadding2D  : (?, 134, 134, 3) (?, 134, 134, 3)
     After Conv2D padding : (?, 64, 64, 64) (?, 64, 64, 64)
     After BatchNorm      : (?, 64, 64, 64) (?, 64, 64, 64)
     C1 Shape: (?, 32, 32, 64) (?, 32, 32, 64)
     C2 Shape:  (?, 32, 32, 256) (?, 32, 32, 256)
     C3 Shape:  (?, 16, 16, 512) (?, 16, 16, 512)
     C4 Shape:  (?, 8, 8, 1024) (?, 8, 8, 1024)
     C5 Shape:  (?, 4, 4, 2048) (?, 4, 4, 2048)

>>> Feature Pyramid Network (FPN) Graph 
     FPN P2 shape : (None, 32, 32, 256)
     FPN P3 shape : (None, 16, 16, 256)
     FPN P4 shape : (None, 8, 8, 256)
     FPN P5 shape : (None, 4, 4, 256)
     FPN P6 shape : (None, 2, 2, 256)

>>> RPN Layer 
  

In [4]:
## Load Mask RCNN Model Weight file
exclude_layers = \
       ['fcn_block1_conv1' 
       ,'fcn_block1_conv2' 
       ,'fcn_block1_pool' 
       ,'fcn_block2_conv1'
       ,'fcn_block2_conv2' 
       ,'fcn_block2_pool'  
       ,'fcn_block3_conv1' 
       ,'fcn_block3_conv2' 
       ,'fcn_block3_conv3' 
       ,'fcn_block3_pool'  
       ,'fcn_block4_conv1' 
       ,'fcn_block4_conv2' 
       ,'fcn_block4_conv3' 
       ,'fcn_block4_pool'  
       ,'fcn_block5_conv1' 
       ,'fcn_block5_conv2' 
       ,'fcn_block5_conv3' 
       ,'fcn_block5_pool'  
       ,'fcn_fc1'          
       ,'dropout_1'        
       ,'fcn_fc2'          
       ,'dropout_2'        
       ,'fcn_classify'     
       ,'fcn_bilinear'     
       ,'fcn_heatmap_norm' 
       ,'fcn_scoring'      
       ,'fcn_heatmap'      
       ,'fcn_norm_loss']
mrcnn_model.load_model_weights( init_with = args.model)   
print('==========================================')
print(" MRCNN MODEL Load weight file COMPLETE    ")
print('==========================================')

mrcnn_config.display()  
mrcnn_model.layer_info()


-----------------------------------------------
 Load model with init parm:  /home/kbardool/models/newshape_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5
 Exclude layers: 
None
-----------------------------------------------
Loading weights from  /home/kbardool/models/newshape_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5

>>> load_weights()
    load_weights: Loading weights from: /home/kbardool/models/newshape_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5
   ----------------
    layers to load 
   ----------------
    >layer 0 : name : input_image                               type: <keras.engine.topology.InputLayer object at 0x7f2dbe6a2be0>
    >layer 1 : name : zero_padding2d_1                          type: <keras.layers.convolutional.ZeroPadding2D object at 0x7f2dbe653ef0>
    >layer 2 : name : conv1                                     type: <keras.layers.convolutional.Conv2D object at 0x7f2dbe658978>
    >layer 3 : name : bn_conv1                                  t

    load_weights: Log directory set to : /home/kbardool/models/newshape_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5
    set_log_dir(): Checkpoint path/filename set to          : /home/kbardool/models/train_fcn_alt/shapes20180621T1554/mask_rcnn_shapes_{epoch:04d}.h5 
    set_log_dir(): Last completed epoch (self.epoch) set to : 1120 
    Load weights complete :  /home/kbardool/models/newshape_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5
 MRCNN MODEL Load weight file COMPLETE    

Configuration Parameters:
-------------------------
BACKBONE_SHAPES                [[32 32]
 [16 16]
 [ 8  8]
 [ 4  4]
 [ 2  2]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     8
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
EARLY_STOP_PATIENCE            80
EPOCHS_TO_RUN                  100
FCN_INPUT_SHAPE                [128 128]
GPU_COUNT                  

In [5]:
##------------------------------------------------------------------------------------
## Build configuration for FCN model
##------------------------------------------------------------------------------------
fcn_config                    = shapes.NewShapesConfig()
fcn_config.BATCH_SIZE         = mrcnn_config.BATCH_SIZE                 # Batch size is 2 (# GPUs * images/GPU).
fcn_config.IMAGES_PER_GPU     = mrcnn_config.BATCH_SIZE               # Must match BATCH_SIZE
fcn_config.STEPS_PER_EPOCH    = int(args.steps_in_epoch)

# fcn_config.LEARNING_RATE      = float(args.lr)
fcn_config.LEARNING_RATE      = 0.005
                          
fcn_config.EPOCHS_TO_RUN      = int(args.epochs)
fcn_config.FCN_INPUT_SHAPE    = mrcnn_config.IMAGE_SHAPE[0:2]
fcn_config.LAST_EPOCH_RAN     = int(args.last_epoch)
fcn_config.WEIGHT_DECAY       = 2.0e-4
fcn_config.VALIDATION_STEPS   = 100
fcn_config.REDUCE_LR_FACTOR   = 0.5
fcn_config.REDUCE_LR_COOLDOWN = 30
fcn_config.REDUCE_LR_PATIENCE = 40
fcn_config.EARLY_STOP_PATIENCE= 80
fcn_config.MIN_LR             = 1.0e-10
fcn_config.display() 



Configuration Parameters:
-------------------------
BACKBONE_SHAPES                [[32 32]
 [16 16]
 [ 8  8]
 [ 4  4]
 [ 2  2]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     8
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
EARLY_STOP_PATIENCE            80
EPOCHS_TO_RUN                  100
FCN_INPUT_SHAPE                [128 128]
GPU_COUNT                      1
IMAGES_PER_GPU                 8
IMAGE_BUFFER                   20
IMAGE_MAX_DIM                  128
IMAGE_MIN_DIM                  128
IMAGE_PADDING                  True
IMAGE_SHAPE                    [128 128   3]
LAST_EPOCH_RAN                 0
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.005
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MAX_SHAPES_PER_IMAGE           15
MEAN_PIXEL              

In [32]:
##------------------------------------------------------------------------------------
## Build FCN Model in Training Mode
##------------------------------------------------------------------------------------
try :
    del fcn_model
    gc.collect()
except: 
    pass
fcn_model = fcn_modellib.FCN(mode="training", config=fcn_config, model_dir=MODEL_DIR)

print(' COCO Model Path       : ', COCO_MODEL_PATH)
print(' Checkpoint folder Path: ', MODEL_DIR)
print(' Model Parent Path     : ', MODEL_PATH)



>>> Initialize FCN model, mode:  training
    set_log_dir(): Checkpoint path/filename set to          : /home/kbardool/models/train_fcn_alt/shapes20180709T1535/fcn_shapes_{epoch:04d}.h5 
    set_log_dir(): Last completed epoch (self.epoch) set to : 0 


---------------------------------------------------
    Adding  FCN layers
---------------------------------------------------

>>> FCN Layer (without L2 Regularization) 
     feature map shape is  (?, 128, 128, 7)
     height : 128 width : 128 classes : 7
     image_data_format:  channels_last
     rois_per_class   :  channels_last
   FCN Block 11 shape is :  (?, 128, 128, 64)
   FCN Block 12 shape is :  (?, 128, 128, 64)
   FCN Block 13 shape is :  (?, 64, 64, 64)
   FCN Block 21 shape is :  (?, 64, 64, 128)
   FCN Block 22 shape is :  (?, 64, 64, 128)
   FCN Block 23 (Max pooling) shape is :  (?, 32, 32, 128)
   FCN Block 31 shape is :  (?, 32, 32, 256)
   FCN Block 32 shape is :  (?, 32, 32, 256)
   FCN Block 33 shape is :  (?, 32, 

In [33]:
print('=====================================')
print(" Load second weight file  ")
print('=====================================')
fcn_model.load_model_weights(init_with = VGG16_MODEL_PATH)

print('=====================================')
print(" Load second weight file COMPLETE    ")
print('=====================================')
fcn_config.display()  
fcn_model.layer_info()



 Load second weight file  
-----------------------------------------------
 Load model with init parm:  /home/kbardool/models/fcn_vgg16_weights_tf_dim_ordering_tf_kernels.h5
 Exclude layers: 
None
-----------------------------------------------
   Loading weights from provided file : /home/kbardool/models/fcn_vgg16_weights_tf_dim_ordering_tf_kernels.h5
>>> load_weights() from : /home/kbardool/models/fcn_vgg16_weights_tf_dim_ordering_tf_kernels.h5
--------------------------------------
 layers to load (not in exclude list) 
--------------------------------------
>layer 0 : name : input_pr_hm_norm                          type: <keras.engine.topology.InputLayer object at 0x7f2d24bb07b8>
>layer 1 : name : fcn_block1_conv1                          type: <keras.layers.convolutional.Conv2D object at 0x7f2d24bb0c18>
>layer 2 : name : fcn_block1_conv2                          type: <keras.layers.convolutional.Conv2D object at 0x7f2d24bb09b0>
>layer 3 : name : fcn_block1_pool                   

In [44]:
train_layers = ['fcn']
loss_names   = ["fcn_norm_loss"]

fcn_model.epoch                  = fcn_config.LAST_EPOCH_RAN
fcn_model.config.LEARNING_RATE   = fcn_config.LEARNING_RATE
fcn_model.config.STEPS_PER_EPOCH = fcn_config.STEPS_PER_EPOCH
batch_size                       = fcn_config.BATCH_SIZE
epochs_to_run = fcn_config.EPOCHS_TO_RUN
learning_rate = fcn_config.LEARNING_RATE
steps_per_epoch = fcn_config.STEPS_PER_EPOCH


train_in_batches_dev(fcn_model,
            mrcnn_model,    
            dataset_train,
            dataset_val, 
            learning_rate = fcn_config.LEARNING_RATE,  
            layers = train_layers,
            losses = loss_names,
            epochs_to_run = fcn_config.EPOCHS_TO_RUN,
            steps_per_epoch = fcn_config.STEPS_PER_EPOCH ,    # gets value form self.config.STEPS_PER_EPOCH
            # epochs = 25,                                      # total number of epochs to run (accross multiple trainings)
            # batch_size = fcn_config.BATCH_SIZE,               # gets value from self.config.BATCH_SIZE
            min_LR          = fcn_config.MIN_LR
            )



['fcn']
['(fcn\\_.*)']
layers regex : (fcn\_.*)
   1  fcn_block1_conv1       (Conv2D              )   TRAIN 
   2  fcn_block1_conv2       (Conv2D              )   TRAIN 
   4  fcn_block2_conv1       (Conv2D              )   TRAIN 
   5  fcn_block2_conv2       (Conv2D              )   TRAIN 
   7  fcn_block3_conv1       (Conv2D              )   TRAIN 
   8  fcn_block3_conv2       (Conv2D              )   TRAIN 
   9  fcn_block3_conv3       (Conv2D              )   TRAIN 
  11  fcn_block4_conv1       (Conv2D              )   TRAIN 
  12  fcn_block4_conv2       (Conv2D              )   TRAIN 
  13  fcn_block4_conv3       (Conv2D              )   TRAIN 
  15  fcn_block5_conv1       (Conv2D              )   TRAIN 
  16  fcn_block5_conv2       (Conv2D              )   TRAIN 
  17  fcn_block5_conv3       (Conv2D              )   TRAIN 
  19  fcn_fc1                (Conv2D              )   TRAIN 
  21  fcn_fc2                (Conv2D              )   TRAIN 
  23  fcn_classify           (Conv2D 

 20/128 [===>..........................] - ETA: 5:30 - loss: 0.0221 - fcn_norm_loss: 0.0221 self.epoch 0   epochs 100  step 20 


KeyboardInterrupt: 

In [43]:
def train_in_batches_dev(self,
          mrcnn_model,
          train_dataset, 
          val_dataset, 
          learning_rate, 
          layers            = None,
          losses            = None,              
          epochs            = 0,
          epochs_to_run     = 1, 
          batch_size        = 0, 
          steps_per_epoch   = 0,
          min_LR            = 0.00001):

    '''
    Train the model.
    train_dataset, 
    val_dataset:    Training and validation Dataset objects.

    learning_rate:  The learning rate to train with

    epochs:         Number of training epochs. Note that previous training epochs
                    are considered to be done already, so this actually determines
                    the epochs to train in total rather than in this particaular
                    call.

    layers:         Allows selecting wich layers to train. It can be:
                    - A regular expression to match layer names to train
                    - One of these predefined values:
                    heads: The RPN, classifier and mask heads of the network
                    all: All the layers
                    3+: Train Resnet stage 3 and up
                    4+: Train Resnet stage 4 and up
                    5+: Train Resnet stage 5 and up
    '''
    assert self.mode == "training", "Create model in training mode."

    if batch_size == 0 :
        batch_size = self.config.BATCH_SIZE

    # if epochs_to_run > 0 :
    epochs = self.epoch + epochs_to_run

    if steps_per_epoch == 0:
        steps_per_epoch = self.config.STEPS_PER_EPOCH

    # use Pre-defined layer regular expressions
    # if layers in self.layer_regex.keys():
        # layers = self.layer_regex[layers]
    print(layers)
    # train_regex_list = []
    # for x in layers:
        # print( ' layers ias : ',x)
        # train_regex_list.append(x)
    train_regex_list = [self.layer_regex[x] for x in layers]
    print(train_regex_list)
    layers = '|'.join(train_regex_list)        
    print('layers regex :', layers)


    ##--------------------------------------------------------------------------------
    ## Data generators
    ##--------------------------------------------------------------------------------
    train_generator = data_generator(train_dataset, self.config, shuffle=True,
                                     batch_size=batch_size)
    val_generator   = data_generator(val_dataset, self.config, shuffle=True,
                                     batch_size=batch_size,
                                     augment=False)

    ##--------------------------------------------------------------------------------
    ## Set trainable layers and compile
    ##--------------------------------------------------------------------------------
    self.set_trainable(layers)            
    self.compile(learning_rate, self.config.LEARNING_MOMENTUM, losses)        


    ##--------------------------------------------------------------------------------
    ## Create checkpoint folder if it doesn't exists
    ##--------------------------------------------------------------------------------
    from tensorflow.python.platform import gfile
    if not gfile.IsDirectory(self.log_dir):
        log('Creating checkpoint folder : {}'.format(self.log_dir))
        gfile.MakeDirs(self.log_dir)
    else:
        log('Checkpoint folder already exists: {}'.format(self.log_dir))                                   
    # my_callback = MyCallback()

    ##--------------------------------------------------------------------------------
    ## Callbacks
    ##--------------------------------------------------------------------------------
    # call back for model checkpoint was originally (?) loss. chanegd to val_loss (which is default) 2-5-18

    # copied from \keras\engine\training.py
    # def _get_deduped_metrics_names(self):
    ## get metrics from keras_model.metrics_names
    out_labels = self.get_deduped_metrics_names()
    print()
    print(' out_labels from get_deduped_metrics_names() : ')
    print(' --------------------------------------------- ')
    print(out_labels)

    ## setup Progress Bar callback
    callback_metrics = out_labels + ['val_' + n for n in out_labels]
    print()
    print(' Callback metrics monitored by progbar :')
    print(' ---------------------------------------')
    pp.pprint(callback_metrics)

    # progbar = keras.callbacks.ProgbarLogger(count_mode='steps')
    # progbar.set_model(self.keras_model)
    # progbar.set_params({
        # 'epochs': epochs,
        # 'steps': steps_per_epoch,
        # 'verbose': 1,
        # 'do_validation': False,
        # 'metrics': callback_metrics,
    # })


    # progbar.set_model(self.keras_model) 

    ## setup Checkpoint callback
    # chkpoint = keras.callbacks.ModelCheckpoint(self.checkpoint_path, 
                                               # monitor='val_loss', 
                                               # verbose=1, 
                                               # save_best_only = True, 
                                               # save_weights_only=True)
    # chkpoint.set_model(self.keras_model)

    # progbar.on_train_begin()



    callbacks_list = [
        keras.callbacks.ProgbarLogger(count_mode='steps'),

        keras.callbacks.TensorBoard(log_dir=self.log_dir,
                                      histogram_freq=0,
                                      batch_size=32,
                                      write_graph=True,
                                      write_grads=False,
                                      write_images=True,
                                      embeddings_freq=0,
                                      embeddings_layer_names=None,
                                      embeddings_metadata=None)

        , keras.callbacks.ModelCheckpoint(self.checkpoint_path, 
                                          mode = 'auto', 
                                          period = 1, 
                                          monitor='val_loss', 
                                          verbose=1, 
                                          save_best_only = True, 
                                          save_weights_only=True)

        , keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                            mode     = 'auto', 
                                            factor   = self.config.REDUCE_LR_FACTOR,   
                                            cooldown = self.config.REDUCE_LR_COOLDOWN,
                                            patience = self.config.REDUCE_LR_PATIENCE,
                                            min_lr   = self.config.MIN_LR, 
                                            verbose  = 1)                                            

        , keras.callbacks.EarlyStopping(monitor='val_loss', 
                                            mode      = 'auto', 
                                            min_delta = 0.00001, 
                                            patience  = self.config.EARLY_STOP_PATIENCE, 
                                            verbose   = 1)                                            
        # , my_callback
    ]


    callbacks =  keras.callbacks.CallbackList(callbacks = callbacks_list)
    callbacks.set_model(self.keras_model)
    callbacks.set_params({
        'epochs': epochs,
        'steps': steps_per_epoch,
        'verbose': 1,
        'do_validation': False,
        'metrics': callback_metrics,
    })



    log("Starting at epoch {} of {} epochs. LR={}\n".format(self.epoch, epochs, learning_rate))
    log("Steps per epochs {} ".format(steps_per_epoch))
    log("    Last epoch completed : {} ".format(self.epoch))
    log("    Starting from epoch  : {} for {} epochs".format(self.epoch, epochs_to_run))
    log("    Learning Rate        : {} ".format(learning_rate))
    log("    Steps per epoch      : {} ".format(steps_per_epoch))
    log("    Batch Size           : {} ".format(batch_size))
    log("    Checkpoint Folder    : {} ".format(self.checkpoint_path))



    ##--------------------------------------------------------------------------------
    ## Start main training loop
    ##--------------------------------------------------------------------------------
    epoch_idx = self.epoch
    # progbar.on_train_begin()
    callbacks.on_train_begin()

    if epoch_idx >= epochs:
        print('Final epoch {} has already completed - Training will not proceed'.format(epochs))
    else:

        while epoch_idx < epochs :
            # progbar.on_epoch_begin(epoch_idx)
            callbacks.on_epoch_begin(epoch_idx)

            for steps_index in range(steps_per_epoch):

                batch_logs = {}
                print(' self.epoch {}   epochs {}  step {} '.format(self.epoch, epochs, steps_index))
                batch_logs['batch'] = steps_index
                batch_logs['size']  = batch_size
                # progbar.on_batch_begin(steps_index, batch_logs)
                callbacks.on_batch_begin(steps_index, batch_logs)

                train_batch_x, train_batch_y = next(train_generator)
                # print('length of train_batch_x:', len(train_batch_x))
                # print('length of train_batch_y:', len(train_batch_y))


                # # model_output   = get_layer_output_2(mrcnn_model.keras_model, train_batch_x, training_flag = False)
                # # model_output = get_layer_output_1(model.keras_model, train_batch_x, [ 26], 1)

                # print(len(model_output))
                # # print(type(output_rois))
                # for i in model_output:
                    # print( i.shape)                    

                results = mrcnn_model.keras_model.predict(train_batch_x)
#                 print('# of items in results:', len(results))


                pr_hm_norm, gt_hm_norm, pr_hm_scores, gt_hm_scores = results[11:]                 

                # print('pr_hm_norm shape   :', pr_hm_norm.shape)
                # print('pr_hm_scores shape :', pr_hm_scores.shape)
                # print('gt_hm_norm shape   :', gt_hm_norm.shape)
                # print('gt_hm_scores shape :', gt_hm_scores.shape)

                outs = self.keras_model.train_on_batch([pr_hm_norm,  pr_hm_scores,gt_hm_norm, gt_hm_scores], train_batch_y)

#                 print(' outs: ', outs)
                if not isinstance(outs, list):
                    outs = [outs]
                for l, o in zip(out_labels, outs):
                    batch_logs[l] = o

                # progbar.on_batch_end(steps_index, batch_logs)
                callbacks.on_batch_end(steps_index, batch_logs)

                # print(outs)

            ## end of epoch operations     
            ##-------------------------------
            val_batch_x, val_batch_y = next(val_generator)
            val_outs = self.keras_model.test_on_batch(X_val, Y_val)
            # write_log(callback, val_names, logs, batch_no//10)
            print(' validation logs output: ', val_outs)
            if not isinstance(val_outs, list):
                val_outs = [val_outs]
            for l, o in zip(out_labels, outs):
                batch_logs[l] = o




            # progbar.on_epoch_end(epoch_idx, {})
            # if (epoch_idx % 10) == 0:
            # chkpoint.on_epoch_end(epoch_idx  , batch_logs)
            callbacks.on_epoch_end(epoch_idx, batch_logs)
            epoch_idx += 1

        ## end of all epochs operations
        ##--------------------------------
        # if epoch_idx != self.epoch:
        # chkpoint.on_epoch_end(epoch_idx -1, batch_logs)
        callbacks.on_train_end()
        self.epoch = max(epoch_idx - 1, epochs)
        print('Final : self.epoch {}   epochs {}'.format(self.epoch, epochs))

    ##--------------------------------------------------------------------------------
    ## End main training loop
    ##--------------------------------------------------------------------------------




In [ ]:
print(train_layers)
# train_regex_list = []
# for x in layers:
    # print( ' layers ias : ',x)
    # train_regex_list.append(x)
train_regex_list = [fcn_model.layer_regex[x] for x in train_layers]
print(train_regex_list)
layers = '|'.join(train_regex_list)        
print('layers regex :', layers)

##--------------------------------------------------------------------------------
## Set trainable layers and compile
##--------------------------------------------------------------------------------
fcn_model.set_trainable(layers)            
fcn_model.compile(learning_rate, fcn_model.config.LEARNING_MOMENTUM, loss_names)   

In [20]:
out_labels = fcn_model.get_deduped_metrics_names()
print()
print(' out_labels from get_deduped_metrics_names() : ')
print(' --------------------------------------------- ')
print(out_labels)


 out_labels from get_deduped_metrics_names() : 
 --------------------------------------------- 
['loss', 'fcn_norm_loss']


In [11]:
##--------------------------------------------------------------------------------
## Data generators
##--------------------------------------------------------------------------------
train_generator = data_generator(dataset_train, mrcnn_model.config, shuffle=True,
                                 batch_size=batch_size)
val_generator   = data_generator(dataset_val, mrcnn_model.config, shuffle=True,
                                 batch_size=batch_size,
                                 augment=False)

In [ ]:
fcn_config.EPOCHS_TO_RUN

In [12]:

epochs = fcn_model.epoch + epochs_to_run
log("Starting at epoch {} of {} epochs. LR={}\n".format(fcn_model.epoch, epochs, learning_rate))
log("Steps per epochs {} ".format(steps_per_epoch))
log("    Last epoch completed : {} ".format(fcn_model.epoch))
log("    Starting from epoch  : {} for {} epochs".format(fcn_model.epoch, epochs_to_run))
log("    Learning Rate        : {} ".format(learning_rate))
log("    Steps per epoch      : {} ".format(steps_per_epoch))
log("    Batch Size           : {} ".format(batch_size))
log("    Checkpoint Folder    : {} ".format(fcn_model.checkpoint_path))

Starting at epoch 0 of 100 epochs. LR=0.005

Steps per epochs 128 
    Last epoch completed : 0 
    Starting from epoch  : 0 for 100 epochs
    Learning Rate        : 0.005 
    Steps per epoch      : 128 
    Batch Size           : 8 
    Checkpoint Folder    : /home/kbardool/models/train_fcn_alt/shapes20180709T1458/fcn_shapes_{epoch:04d}.h5 


In [24]:
steps_index = 0 

In [25]:
batch_logs = {}
print(' self.epoch {}   epochs {}  step {} '.format(fcn_model.epoch, epochs, steps_index))
batch_logs['batch'] = steps_index
batch_logs['size']  = batch_size
# progbar.on_batch_begin(steps_index, batch_logs)
callbacks.on_batch_begin(steps_index, batch_logs)

 self.epoch 0   epochs 100  step 0 


NameError: name 'callbacks' is not defined

In [13]:
train_batch_x, train_batch_y = next(train_generator)

for i in train_batch_x:
    print( i.shape)
  

(8, 128, 128, 3)
(8, 15)
(8, 4092, 1)
(8, 256, 4)
(8, 100)
(8, 100, 4)
(8, 56, 56, 100)


### Run MRCNN model predictions

     Outputs:
     --------
     layer:  0    output : rpn_class_logits/rpn_class_logits:0        Type: float32           Shape: (?, ?, 2)
     layer:  1    output : rpn_class/rpn_class:0                      Type: float32           Shape: (?, ?, 2)
     layer:  2    output : rpn_bbox/rpn_bbox:0                        Type: float32           Shape: (?, ?, 4)
     layer:  3    output : rpn_proposal_rois/rpn_roi_proposals:0      Type: float32           Shape: (8, ?, ?)
     layer:  4    output : proposal_targets/output_rois:0             Type: float32           Shape: (8, ?, ?)
     layer:  5    output : proposal_targets/target_class_ids:0        Type: int32             Shape: (8, ?)
     layer:  6    output : proposal_targets/target_bbox_deltas:0      Type: float32           Shape: (8, ?, ?)
     layer:  7    output : proposal_targets/roi_gt_boxes:0            Type: float32           Shape: (8, ?, ?)
     layer:  8    output : mrcnn_class_logits/mrcnn_class_logits:0    Type: float32           Shape: (?, 32, 7)
     layer:  9    output : mrcnn_class/mrcnn_class:0                  Type: float32           Shape: (?, 32, 7)
     layer: 10    output : mrcnn_bbox_regression/mrcnn_bbox:0         Type: float32           Shape: (?, 32, 7, 4)
     layer: 11    output : cntxt_layer/pred_heatmap_norm:0            Type: float32           Shape: (8, 128, 128, 7)
     layer: 12    output : cntxt_layer/gt_heatmap_norm:0              Type: float32           Shape: (8, 128, 128, 7)
     layer: 13    output : cntxt_layer/pred_heatmap_scores:0          Type: float32           Shape: (?, ?, ?, ?)
     layer: 14    output : cntxt_layer/gt_heatmap_scores:0            Type: float32           Shape: (?, ?, ?, ?)
        

     Inputs:
     -------
     layer:  0    output : input_pr_hm_norm:0                         Type: float32           Shape: (?, 128, 128, 7)
     layer:  1    output : input_pr_hm_scores:0                       Type: float32           Shape: (?, 7, 32, 11)
     layer:  2    output : input_gt_hm_norm:0                         Type: float32           Shape: (?, 128, 128, 7)
     layer:  3    output : input_gt_hm_scores:0                       Type: float32           Shape: (?, 7, 32, 11)


     Outputs:
     --------
     layer:  0    output : fcn_heatmap_norm/fcn_heatmap_norm:0        Type: float32           Shape: (?, 128, 128, 7)
     layer:  1    output : fcn_scoring/fcn_heatmap_scores:0           Type: float32           Shape: (?, ?, ?, ?)
     layer:  2    output : fcn_heatmap/fcn_heatmap:0                  Type: float32           Shape: (?, 128, 128, 7)
     layer:  3    output : fcn_norm_loss/fcn_norm_loss:0              Type: float32           Shape: (1, 1)     

In [14]:
results = mrcnn_model.keras_model.predict(train_batch_x)
print('# of items in results:', len(results))

for i in results:
    print( i.shape)


# of items in results: 15
(8, 4092, 2)
(8, 4092, 2)
(8, 4092, 4)
(8, 1000, 4)
(8, 32, 4)
(8, 32)
(8, 32, 4)
(8, 32, 4)
(8, 32, 7)
(8, 32, 7)
(8, 32, 7, 4)
(8, 128, 128, 7)
(8, 128, 128, 7)
(8, 7, 32, 11)
(8, 7, 32, 11)


In [15]:
pr_hm_norm, gt_hm_norm, pr_hm_scores, gt_hm_scores = results[11:]                 
print(pr_hm_norm.shape)
print(gt_hm_norm.shape)
print(pr_hm_scores.shape)
print(gt_hm_scores.shape)

(8, 128, 128, 7)
(8, 128, 128, 7)
(8, 7, 32, 11)
(8, 7, 32, 11)


In [16]:
outs = fcn_model.keras_model.train_on_batch([pr_hm_norm,  pr_hm_scores,gt_hm_norm, gt_hm_scores], train_batch_y)

  Make training function -------------------------------
 self.total_loss : loss_1/add_1:0
<tf.Tensor 'loss_1/add_1:0' shape=(1, 1) dtype=float32>
 self.metrics_tensors : Mean_3:0
[<tf.Tensor 'Mean_3:0' shape=(1, 1) dtype=float32>]
 updates : 
[   <tf.Tensor 'training/SGD/AssignAdd:0' shape=() dtype=int64_ref>,
    <tf.Tensor 'training/SGD/Assign:0' shape=(3, 3, 7, 64) dtype=float32_ref>,
    <tf.Tensor 'training/SGD/Assign_1:0' shape=(3, 3, 7, 64) dtype=float32_ref>,
    <tf.Tensor 'training/SGD/Assign_2:0' shape=(64,) dtype=float32_ref>,
    <tf.Tensor 'training/SGD/Assign_3:0' shape=(64,) dtype=float32_ref>,
    <tf.Tensor 'training/SGD/Assign_4:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
    <tf.Tensor 'training/SGD/Assign_5:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
    <tf.Tensor 'training/SGD/Assign_6:0' shape=(64,) dtype=float32_ref>,
    <tf.Tensor 'training/SGD/Assign_7:0' shape=(64,) dtype=float32_ref>,
    <tf.Tensor 'training/SGD/Assign_8:0' shape=(3, 3, 64, 128) dtype=fl

In [21]:
pp.pprint(outs)
print(' outs: ', outs)
if not isinstance(outs, list):
    outs = [outs]
for l, o in zip(out_labels, outs):
    batch_logs[l] = o


[array([[0.0041]], dtype=float32), array([[0.0041]], dtype=float32)]
 outs:  [array([[0.0041]], dtype=float32), array([[0.0041]], dtype=float32)]


NameError: name 'batch_logs' is not defined

In [ ]:
fcn_output = get_layer_output_2(fcn_model.keras_model, [pr_hm_norm,  pr_hm_scores,gt_hm_norm, gt_hm_scores], True)


In [21]:
fcn_model.keras_model.metrics_names

['loss', 'fcn_norm_loss']

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

### Training FPN, RPN and MRCNN heads using  Keras.model.fit_generator()

print(config.BATCH_SIZE)
print(model.config.BATCH_SIZE)
print(model.config.LEARNING_RATE)
import keras
keras.__version__

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.

# model.train(dataset_train, dataset_val, 
#             learning_rate=config.LEARNING_RATE, 
# #             epochs = 69,
#             epochs_to_run =2, 
#             layers='heads')
## Last run prior to FCN training was 3699, last checkpoint was 3892  ...start at 3899

train_layers = [ 'mrcnn', 'fpn','rpn']
loss_names   = [ "rpn_class_loss", "rpn_bbox_loss" , "mrcnn_class_loss", "mrcnn_bbox_loss"]
model.epoch = 1233
model.config.LEARNING_RATE = 1.0e-4
model.config.STEPS_PER_EPOCH = 7

model.train(dataset_train, dataset_val, 
            learning_rate=model.config.LEARNING_RATE, 
            epochs_to_run =3000, 
#             epochs = 25,            
#             batch_size = 0
#             steps_per_epoch = 0 
            layers = train_layers,
            losses = loss_names,
            min_LR = 1.0e-6,
            )

###  Train FCN head layers

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.

# model.train(dataset_train, dataset_val, 
#             learning_rate=config.LEARNING_RATE, 
# #             epochs = 69,
#             epochs_to_run =2, 
#             layers='heads')

## Last run prior to FCN training was 3699, last checkpoint was 3892

train_layers = ['fcn']
loss_names   = [  "fcn_norm_loss"]
model.epoch = 1668
model.config.LEARNING_RATE = 1.0e-4
model.config.STEPS_PER_EPOCH = 8 

model.train(dataset_train, dataset_val, 
            learning_rate=model.config.LEARNING_RATE, 
            epochs_to_run = 500, 
#             epochs = 25,            
#             batch_size = 6,
#             steps_per_epoch = 0 
            layers = train_layers,
            losses = loss_names,
            min_LR = 1.0e-9
            )

In [ ]:
model.keras_model.losses
print(model.keras_model.metrics_names)

## - Training heads using train_in_batches ()

We need to use this method for the time being as the fit generator does not have provide EASY access to the output in Keras call backs. By training in batches, we pass a batch through the network, pick up the generated RoI detections and bounding boxes and generate our semantic / gaussian tensors ...


In [ ]:
model.train_in_batches(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE/6, 
            epochs_to_run = 3,
            layers='heads')

## Fine Tuning
Fine tune all layers

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=211,
            layers="all")

# train_layers = ['fcn']
# loss_names   = [  "fcn_norm_loss"]
# model.epoch = 208
# model.config.LEARNING_RATE = 1.0e-4
# model.config.STEPS_PER_EPOCH = 8 

# model.train(dataset_train, dataset_val, 
#             learning_rate=model.config.LEARNING_RATE, 
#             epochs_to_run = 500, 
# #             epochs = 25,            
# #             batch_size = 6,
# #             steps_per_epoch = 0 
#             layers = train_layers,
#             losses = loss_names,
#             min_LR = 1.0e-7
#             )

## Save 

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes_post_training.h5")
model.keras_model.save_weights(model_path)

In [ ]:
# model.keras_model.summary(line_length=132, positions=[0.30,0.75, .83, 1. ])

In [ ]:
train_batch_x, train_batch_y = next(train_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [ ]:
model.layer_info()

In [ ]:
# train_batch_x, train_batch_y = next(train_generator)

model_output = get_layer_output_2(model.keras_model, train_batch_x, 1)
# model_output = get_layer_output_1(model.keras_model, train_batch_x, [ 26], 1)

print(len(model_output))

# rpn_class_loss            = model_output[0]          # layer: 11   shape: (1, 1)
# rpn_bbox_loss             = model_output[1]          # layer: 12   shape: (1, 1)
# mrcnn_class_loss          = model_output[2]          # layer: 13   shape: (1, 1)
# mrcnn_bbox_loss           = model_output[3]          # layer: 14   shape: (1, 1)
# fcn_normalized_loss       = model_output[0]          # layer: 26   shape: (1, 1)

# print(type(output_rois))
for i in model_output:
    print( i.shape)
# print('FCN Normalized Loss is :', fcn_normalized_loss)    


In [ ]:
input_image      =  train_batch_x[0]
input_image_meta =  train_batch_x[1]
input_rpn_match  =  train_batch_x[2]
input_rpn_bbox   =  train_batch_x[3]
input_gt_class_ids = train_batch_x[4]
input_gt_bboxes    = train_batch_x[5]
input_gt_masks     = train_batch_x[6]
print(' Input image shape is :', input_image.shape)
h, w = input_image.shape[1], input_image.shape[2]      #  tf.shape(input_image)[1], tf.shape(input_image)[2]
input_normlzd_gt_bboxes = tf.identity(input_gt_bboxes / [h,w,h,w])

# gt_masks   =  train_batch_x[6]
print(' input_rpn_match    ', input_rpn_match.shape)
print(' input_rpn_bbox     ', input_rpn_bbox.shape)
print(' input_gt_class_ids ', input_gt_class_ids.shape)
print(' input_gt_bboxes    ', input_gt_bboxes.shape)
print(' input_normlzd_gt_bboxes    ', input_normlzd_gt_bboxes.shape)
